Activate  Hyper-V funtionalities

In [8]:
Enable-WindowsOptionalFeature -Online -FeatureName Microsoft-Hyper-V -All



Path          : 
Online        : True
RestartNeeded : False





After the restart, open the Hyper-V Manager

In [9]:
virtmgmt.msc

Create a new bridge switch

In [13]:
Get-NetAdapter


Name                      InterfaceDescription                    ifIndex Status       MacAddress             LinkSpeed
----                      --------------------                    ------- ------       ----------             ---------
Ethernet                  Realtek PCIe GbE Family Controller           19 Up           4C-CC-6A-62-BB-29         1 Gbps
Ethernet 2                Dell GigabitEthernet                         18 Disconnected 9C-EB-E8-3E-66-13         1 Gbps
vEthernet (vExternalSw... Hyper-V Virtual Ethernet Adapter #2           7 Up           4C-CC-6A-62-BB-29         1 Gbps


vEthernet (Default Swi... Hyper-V Virtual Ethernet Adapter             28 Up           00-15-5D-A7-BF-9F        10 Gbps




In [11]:
$VMSwitchName = 'vExternalSwitch'

# use Get-NetAdapter to get names for the following items if they are eventualy changed on your system
$NetAdapterName = 'Ethernet'

# Warning this will break your ethernet connection during a brief period
New-VMSwitch -Name $VMSwitchName -NetAdapterName $NetAdapterName -AllowManagementOS $true

New-VMSwitch : Failed while adding virtual Ethernet switch connections.
External Ethernet adapter 'Realtek PCIe GbE Family Controller' is already bound to the Microsoft Virtual Switch 
protocol.
At line:6 char:1
+ New-VMSwitch -Name $VMSwitchName -NetAdapterName $NetAdapterName -All ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidArgument: (:) [New-VMSwitch], VirtualizationException
    + FullyQualifiedErrorId : InvalidParameter,Microsoft.HyperV.PowerShell.Commands.NewVMSwitch
 


Create VM

In [33]:
$VMName = 'WS2019'
$VMSwitchName = 'vExternalSwitch'
$VMDiskPath = 'F:\VMs\'

$ISOfile = 'en_windows_server_2019_updated_feb_2021_x64_dvd_277a6bfe.iso'
$BootISOPath = 'G:\GoToBackup\ISO\'

$ProcessorCount = 2
$MemoryStartupBytes = 2gb
$MemoryMinimumBytes = 512mb
$MemoryMaximumBytes = 4gb
$DataDiskSize = 60gb

$BootISOFullPath = $BootISOPath + $ISOfile
$NewVHDPath = $VMDiskPath + $VMName + '.vhdx'

# Clean up
Remove-VM -Name $VMName -Force
Remove-Item $NewVHDPath


New-VM -Name $VMName -Generation 2 -SwitchName $VMSwitchName -NewVHDPath $NewVHDPath -NewVHDSizeBytes $DataDiskSize

Set-VMProcessor -VMName $VMName -Count $ProcessorCount 
Set-VMMemory -VMName $VMName -StartupBytes $MemoryStartupBytes -DynamicMemoryEnabled $true -MinimumBytes $MemoryMinimumBytes -MaximumBytes $MemoryMaximumBytes

Add-VMDvdDrive -VMName $VMName -Path $BootISOFullPath

Set-VMFirmware -VMName $VMName -FirstBootDevice $(Get-VMDvdDrive -VMName $VMName)

Name   State CPUUsage(%) MemoryAssigned(M) Uptime   Status             Version
----   ----- ----------- ----------------- ------   ------             -------
WS2019 Off   0           0                 00:00:00 Operating normally 9.0    




In [34]:
$VMName = 'WS2019'

Start-VM -VMName $VMName

Create checkpoint

In [35]:
$VMName = 'WS2019'
$CheckpointName = 'Clean machine with latest updates'

Checkpoint-VM -Name $VMName -SnapshotName $CheckpointName

Export VM

In [ ]:
$VMName = 'WS2019'
$ExportPath = 'F:\ExportedVMs'

Export-VM -Name $VMName -Path $ExportPath